In [ ]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import rebound
import numpy as np
import random
import math

In [ ]:
# import excel spreadsheet containing stellar masses of Kepler targets
masses = pd.read_excel('masses.xlsx', sheet_name='Sheet1')

# constants
year = 1./365.25 # Julian year
solar_radius_to_au = 0.00465 # AU

In [ ]:
# specify number of systems we want to generate and evolve
number_of_systems = 

# create lists that will store information of the generate circumbinary systems
signature = [] # transit counts per epoch
number_periods_in_window_range = [] # number of epoches a systems evolves through in one Kepler window
abin_range = [] # separation of the binary stars

mu_bin_range = [] # mass ratio of the binary stars
inc_planet_range = [] # inclination of the planetary orbit
inc_binary_range = [] # inclination of the binary orbit
average_count_range = [] # average count per epoch


for i in range(number_of_systems):
    # pull out a stellar mass from the spreadsheet imported above as the mass of one of the binary stars in the system
    index_A = random.randint(0,196467)
    mA = masses['Stellar Masses'][index_A]
    
    # sample the binary mass ratio according to the q**0.5 distribution (Parker & Reggiani 2013)
    temp_r = 0.0 + (random.uniform(0,1)*(1.0-0.0))
    mu_bin = (-math.sqrt(0.1)*(temp_r - 1) + temp_r)**2 
    # calculate the mass of the other star from the binary mass ratio
    mB = mA*mu_bin

    # calculate the stellar radii
    if (mA <= 1.66):
        rA = 1.06*(mA**0.945)*solar_radius_to_au
    else:
        rA = 1.33*(mA**0.555)*solar_radius_to_au
    if (mB <= 1.66):
        rB = 1.06*(mB**0.945)*solar_radius_to_au
    else:
        rB = 1.33*(mB**0.555)*solar_radius_to_au
     
    # assume that both the planetary and binary orbits are circular
    ebin = 0
    eb = 0
    
    # mass ratio of the binary stars as defined by Holman & Wiegert 1998
    mu_bin_prime = (mB)/(mA+mB) 
    
    # now choose a value for the binary separation
    # define minimum possible binary separation
    abin_min = 0.05
    # define maximum binary separation as the separation such that a planet just outside the instability region defined by Holman & Wiegert 1998 has a period of exactly one Kepler window
    Pb_max = 4.35 
    ab_max = (((Pb_max**2)*39.319*(mA+mB))/(4*(math.pi)**2))**(1/3)
    abin_max = ab_max/(1.60 + 5.10*ebin - 2.22*(ebin**2) + 4.12*mu_bin_prime - 4.27*mu_bin_prime*ebin - 5.09*(mu_bin_prime**2) + 4.61*(mu_bin_prime**2)*(ebin**2)) # given by Holman & Wiegert 1998
    # sample a binary separation between min and max according to a log uniform distribution
    abin = math.exp(math.log(abin_min) + (random.uniform(0,1)*(math.log(abin_max)-math.log(abin_min))))

    # now choose a value for the semi-major axis of the planet
    # minimum possible semi-major axis (given the binary separation) is given by the instability region defined by Holman & Wiegert 1998
    ab_min = abin*(1.60 + 5.10*ebin - 2.22*(ebin**2) + 4.12*mu_bin_prime - 4.27*mu_bin_prime*ebin - 5.09*(mu_bin_prime**2) + 4.61*(mu_bin_prime**2)*(ebin**2))
    if (ab_min > ab_max):
        continue
    # remember that maximum possible semi-major axis has a period of exactly one Kepler window
    # sample a semi-major between min and max according to a log uniform distribution
    ab = math.exp(math.log(ab_min) + (random.uniform(0,1)*(math.log(ab_max)-math.log(ab_min))))

    # calculate the period of the planet according to Kepler's 3rd law
    Pb = ((4*(math.pi)**2)*(ab**3)/(39.319*(mA+mB)))**0.5

    # define the radius of the planet to be equal to that of the Earth
    radius_p = 4.24*(10**(-5))
    
    # select values for cos(inc_bin) and Omega_bin such that the binary star orbit is isotropic
    cos_inc_bin_max = 1
    cos_inc_bin_min = -1
    cos_inc_bin = cos_inc_bin_min + (random.uniform(0,1)*(cos_inc_bin_max - cos_inc_bin_min))
    inc_bin = math.acos(cos_inc_bin)
    
    ascending_node_bin = random.uniform(-np.pi/2, np.pi/2)
    
    # select value for cos(inc_p) so that it follows a truncated normal distribution centered at the configuration that's coplanar to the 
    # binary star
    from scipy.stats import truncnorm
    # specify the standard deviation of the truncated normal distribution
    sd = 
    
    distribution = truncnorm(-1.0/sd, 1.0/sd, scale=sd)
    cos_relative_inc = distribution.rvs()
    relative_inc = math.acos(cos_relative_inc)
    inc_p = inc_bin - (math.pi/2 - relative_inc)

    # set inclination and ascending node of the planet relative to the binary orbit, and make them follow a truncated normal distribution.
    distribution = truncnorm(-1.0/sd, 1.0/sd, scale=sd)
    relative_ascending_node = distribution.rvs()*(math.pi/2)
    ascending_node_p = ascending_node_bin - relative_ascending_node
    
    # specify number of planet periods we want to evolve this particular system
    n_periods = 100

    # input all the parameters to rebound
    sim = rebound.Simulation()
    sim.units = ('yr','AU','Msun') 
    sim.G = 39.319
    sim.add(m=mA)
    sim.add(m=mB, a=abin, e=ebin, inc=inc_bin, Omega=ascending_node_bin)
    sim.add(a=ab, e=eb, inc=inc_p, Omega=ascending_node_p) # no mass for the planet
    sim.integrator = "whfast"
    per_p_timestep = 10000 
    sim.dt = Pb/per_p_timestep
    sim.move_to_com()

    # record the positions of the two stars and the planet at each timestep
    Noutputs = int(per_p_timestep*n_periods)
    times = np.linspace(0.,Pb*n_periods,Noutputs)
    x_0 = np.zeros(Noutputs)
    y_0 = np.zeros(Noutputs)
    z_0 = np.zeros(Noutputs)
    x_1 = np.zeros(Noutputs)
    y_1 = np.zeros(Noutputs)
    z_1 = np.zeros(Noutputs)
    x_2 = np.zeros(Noutputs)
    y_2 = np.zeros(Noutputs)
    z_2 = np.zeros(Noutputs)

    particles = sim.particles
    for j,k in enumerate(times):
        sim.integrate(k, exact_finish_time=0)
        x_0[j] = particles[0].x
        y_0[j] = particles[0].y
        z_0[j] = particles[0].z
        x_1[j] = particles[1].x
        y_1[j] = particles[1].y
        z_1[j] = particles[1].z
        x_2[j] = particles[2].x
        y_2[j] = particles[2].y
        z_2[j] = particles[2].z

    # first record the transits of the two stars separately
    transit_a = [0] * n_periods 
    transit_b = [0] * n_periods 

    d_ap = 0 
    d_bp = 0 

    transiting_a = 0 
    transiting_b = 0 

    # function for calculating the euclidean distance
    def eucdist(x1, x2, y1, y2):
        d = ((x2-x1)**2 + (y2-y1)**2)**0.5
        return d
    
    # record a new transit if the planet is in front of a star, projected distance between the star and the planet on the x-y plane is 
    # smaller than the sum of their radii, and the planet is not already transiting in the previous timestep.
    for j in range(0,Noutputs): 
            d_ap = eucdist(x_0[j], x_2[j], y_0[j], y_2[j]) 
            d_bp = eucdist(x_1[j], x_2[j], y_1[j], y_2[j]) 
            for k in range(0, n_periods):
                if (k * per_p_timestep <= j and j < (k+1) * per_p_timestep):
                    if (d_ap <= rA + radius_p and z_2[j] >= z_0[j] and transiting_a == 0 and (z_0[i] >= z_1[i] or (z_0[i] <= z_1[i] and d_bp >= rB + radius_p))):
                        transit_a[k] += 1
                        transiting_a = 1
                    elif (d_ap <= rA + radius_p and z_2[j] >= z_0[j] and transiting_a == 1):
                        transiting_a = 1
                    else:
                        transiting_a = 0

                    if (d_bp <= rB + radius_p and z_2[j] >= z_1[j] and transiting_b == 0 and (z_1[i] >= z_0[i] or (z_1[i] <= z_0[i] and d_ap >= rA + radius_p))):
                        transit_b[k] += 1
                        transiting_b = 1
                    elif (d_bp <= rB + radius_p and z_2[j] >= z_1[j] and transiting_b == 1):
                        transiting_b = 1
                    else:
                        transiting_b = 0
    # calculate the total transits because in real light curve data, we cannot tell the different between the two stars
    total_transit = []                  
    for k in range(len(transit_a)):
        total_transit.append(transit_a[k]+transit_b[k])
        
    # calculate the number of planet periods in one Kepler window
    number_periods_in_window = (4.35//Pb) + 1 
    number_periods_in_window = int(number_periods_in_window)
    if (number_periods_in_window < 1):
        continue # period of such systems is too long to be observable
    if (number_periods_in_window > (4.35*365/50)): # filter out planets with periods shorter than 50 days
        continue

    signature.append(total_transit)
    number_periods_in_window_range.append(number_periods_in_window)
    abin_range.append(abin)
    mu_bin_range.append(mu_bin)
    inc_planet_range.append(inc_p)
    inc_binary_range.append(inc_bin)
    average_count_range.append(1.0*sum(total_transit)/len(total_transit))


print (signature)
print (number_periods_in_window_range)
print (abin_range)
print (mu_bin_range)
print (inc_planet_range)
print (inc_binary_range)
print (average_count_range)